In [7]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import time
import sys
from datetime import datetime
from datetime import date
import urllib.parse

In [8]:
def getAndParseURL(url):
    result = requests.get(url,headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup

In [3]:
book_link = []

html = getAndParseURL("https://www.kitapyurdu.com/kategori/kitap-edebiyat/128.html")
html = getAndParseURL("https://www.kitapyurdu.com/kategori/kitap-cocuk-kitaplari/2.html")
#https://www.kitapyurdu.com/kategori/kitap-sinavlar/737.html
#https://www.kitapyurdu.com/kategori/kitap-kisisel-gelisim/341.html
#https://www.kitapyurdu.com/kategori/kitap-bilgisayar/31.html
#https://www.kitapyurdu.com/kategori/kitap-kultur/77.html
#https://www.kitapyurdu.com/kategori/kitap-tarih/41.html
#https://www.kitapyurdu.com/kategori/kitap-felsefedusunce/424.html
#https://www.kitapyurdu.com/kategori/kitap-psikoloji/87.html
for links in html.findAll("div",{"class":"cover"}):
    book_link.append(links.a["href"])

In [4]:
book_link

['https://www.kitapyurdu.com/kitap/seker-portakali-ciltsiz/10139.html&path=2',
 'https://www.kitapyurdu.com/kitap/benim-zurafam-ucabilir/589761.html&path=2',
 'https://www.kitapyurdu.com/kitap/delifisek/13565.html&path=2',
 'https://www.kitapyurdu.com/kitap/pal-sokagi-cocuklari/125370.html&path=2',
 'https://www.kitapyurdu.com/kitap/gunesi-uyandiralim/13562.html&path=2',
 'https://www.kitapyurdu.com/kitap/babaannem-geri-dondu/578116.html&path=2',
 'https://www.kitapyurdu.com/kitap/martiya-ucmayi-ogreten-kedi/16956.html&path=2',
 'https://www.kitapyurdu.com/kitap/kitaplardan-korkan-cocuk/40901.html&path=2',
 'https://www.kitapyurdu.com/kitap/101-deyim-101-oyku/392721.html&path=2',
 'https://www.kitapyurdu.com/kitap/zaman-bisikleti/57439.html&path=2',
 'https://www.kitapyurdu.com/kitap/101-atasozu-101-oyku/392720.html&path=2',
 'https://www.kitapyurdu.com/kitap/insan-vucudu-tiyatrosu/411661.html&path=2',
 'https://www.kitapyurdu.com/kitap/kraliceyi-kurtarmak/58695.html&path=2',
 'https:/

In [5]:
result = []
for details in book_link[::]:
    html = getAndParseURL(details)
    
    Book_name = html.find("h1",{"class":"pr_header__heading"}).text.strip()
    Writer = html.find("a",{"class":"pr_producers__link"}).text.strip()
    Publisher = html.find("div",{"class":"pr_producers__publisher"}).text.strip()
    Number_of_comments = html.find("p",{"class":"pr_view-review-text"}).findNext().findNext().text.replace(".","").strip()
    
    Sales_amount = html.find("p",{"class":"purchased"})
    if(Sales_amount != None):
        Sales_amount1 = Sales_amount.text.replace("Bu","").replace("üründen","").replace("adet","").replace("satın ","").replace("alınmıştır.","").strip()
    else:
        Sales_amount1 = "0"
    
    Price = html.find("div",{"class":"price__item"}).text.replace(",",".").strip()
    
    Translator = html.find("div",{"class":"attributes"}).find(text = "Çevirmen:")
    if(Translator != None):
        Translator1 = Translator.findNext().text.strip()
    else:
        Translator1 = "NULL"
        
    Book_cover = html.find("div",{"class":"attributes"}).find(text = "Kapak:")
    if(Book_cover != None):
        Book_cover1 = Book_cover.findNext().text.strip()
    else:
        Book_cover1 = "NULL"
    
    Release_date = html.find("div",{"class":"attributes"}).find(text = "Yayın Tarihi:").findNext().text.strip()
    
    ISBN = html.find("div",{"class":"attributes"}).find(text = "ISBN:").findNext().text.strip()

    Language = html.find("div",{"class":"attributes"}).find(text = "Dil:").findNext().text.strip()
       
    Number_of_pages = html.find("div",{"class":"attributes"}).find(text = "Sayfa Sayısı:").findNext().text.strip()

    Dimension = html.find("div",{"class":"attributes"}).find(text = "Boyut:")
    if(Dimension != None):
        Dimension1 = Dimension.findNext().text.replace("cm","").replace("x","").strip()
    else:
        Dimension1 = "NULL"
    
    Skin_type = html.find("div",{"class":"attributes"}).find(text = "Cilt Tipi:").findNext().text.strip()
    
    Paper_type = html.find("div",{"class":"attributes"}).find(text = "Kağıt Cinsi:").findNext().text.strip()
    
    result.append([Book_name,Writer,Publisher,Number_of_comments,Sales_amount1,Price,Translator1,Book_cover1,Release_date,ISBN,Language,
                   Number_of_pages,Dimension1,Skin_type,Paper_type])
columns_name = ["Isim","Yazar","Yayimci","Yorum_Sayisi","Satis_Miktari","Fiyat","Cevirmen","Kapak","Yayin_Tarihi","ISBN","Dil",
                    "Sayfa_Sayisi","Boyut","Cilt_Tipi","Kagit_Cinsi"]
df = pd.DataFrame.from_records(result,columns = columns_name) 

In [6]:
df

,Isim,Yazar,Yayimci,Yorum_Sayisi,Satis_Miktari,Fiyat,Cevirmen,Kapak,Yayin_Tarihi,ISBN,Dil,Sayfa_Sayisi,Boyut,Cilt_Tipi,Kagit_Cinsi
0,Şeker Portakalı (Ciltsiz),Jose Mauro De Vasconcelos,CAN YAYINLARI,9912,260.719,23.29,Emrah İmre,Utku Lomlu,06.09.2019,9789750738609,TÜRKÇE,182,12.5 19.5,Karton Kapak,Kitap Kağıdı
1,Benim Zürafam Uçabilir,Mert Arık,TİMAŞ ÇOCUK YAYINLARI,196,22.314,16.98,NULL,NULL,14.10.2021,9786050842159,TÜRKÇE,64,12.5 19.5,Karton Kapak,Kitap Kağıdı
2,Delifişek,Jose Mauro De Vasconcelos,CAN YAYINLARI,1540,56.836,13.46,İnci Kut,NULL,06.08.2019,9789750724787,TÜRKÇE,88,12.5 19.5,Karton Kapak,3. Hm. Kağıt
3,Pal Sokağı Çocukları,Ferenc Molnar,YAPI KREDİ YAYINLARI,1448,59.816,18.98,Tarık Demirkan,NULL,08.10.2021,9789750815546,TÜRKÇE,235,13.5 19.5,Karton Kapak,Kitap Kağıdı
4,Güneşi Uyandıralım,Jose Mauro De Vasconcelos,CAN YAYINLARI,1843,65.308,24.92,Saadet Özen,NULL,06.08.2019,9789750739071,TÜRKÇE,272,12.5 19.5,Karton Kapak,Kitap Kağıdı
5,Babaannem Geri Döndü,Şermin Yaşar,TAZE KİTAP,156,12.492,40.30,NULL,NULL,12.04.2021,9786057480927,TÜRKÇE,160,13.5 21,Karton Kapak,Kitap Kağıdı
6,Martıya Uçmayı Öğreten Kedi,Luis Sepulveda,CAN ÇOCUK YAYINLARI,314,30.991,16.76,Saadet Özen,NULL,24.09.2021,9789755109428,TÜRKÇE,128,12.5 19.5,Karton Kapak,Kitap Kağıdı
7,Kitaplardan Korkan Çocuk,Susanna Tamaro,CAN ÇOCUK YAYINLARI,721,47.799,12.94,Eren Cendey,NULL,31.10.2019,9789758440863,TÜRKÇE,44,12.5 19.5,Karton Kapak,Kitap Kağıdı
8,101 Deyim 101 Öykü,Süleyman Bulut,CAN ÇOCUK YAYINLARI,298,26.849,22.00,NULL,NULL,19.03.2019,9789750724718,TÜRKÇE,204,16 24,Karton Kapak,Kitap Kağıdı
9,Zaman Bisikleti,Bilgin Adalı,CAN ÇOCUK YAYINLARI,347,34.364,13.22,NULL,NULL,06.07.2021,9789750703508,TÜRKÇE,128,12.5 19.5,Karton Kapak,Kitap Kağıdı
